In [7]:
from collections import Counter
import pandas as pd
from sklearn.model_selection import cross_val_score
import numpy as np
import nltk
import re

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
classificacoes = pd.read_csv('base_15800.csv', encoding='utf-8', error_bad_lines=False, sep=';')
#pegando parte dos dados randomicamente
classificacoes = classificacoes.sample(10000)
textos_puros = classificacoes['andamentos']
frases = textos_puros.str.lower()
textos_quebrados = [nltk.tokenize.word_tokenize(frase) for frase in frases]

In [3]:
print(len(textos_quebrados))
print (textos_quebrados[2])

10000
['25/04/2019', 'recebidos', 'os', 'autos', 'juntar', 'documentos', '29/01/2019', 'recebidos', 'os', 'autos', 'movimentado', 'o', 'apenso', '049/1.19.0000207-1', '29/01/2019', 'recebidos', 'os', 'autos', 'cumprir', 'despacho', '29/01/2019', 'juntada', 'de', 'peticao', 'de', '(', 'outras', ')', 'pelo', 'réu', '25/01/2019', 'recebidos', 'os', 'autos', 'juntar', 'documentos', '24/01/2019', 'documento', '(', 's', ')', 'recebido', '(', 's', ')', 'no', 'protocolo', 'geral', '05/12/2018', 'juntada', 'de', 'aviso', 'de', 'recebimento', '(', 'ar', ')', '-', 'citação', '04/12/2018', 'recebidos', 'os', 'autos', '04/12/2018', 'autos', 'entregues', 'em', 'carga', 'ao', 'para', 'cópia', '-', 'designação', ':', 'cópia', '-', '84376/rs', '29/11/2018', 'recebidos', 'os', 'autos', 'juntar', 'documentos', '21/11/2018', 'recebidos', 'os', 'autos', 'aguarda', 'devolução', 'de', 'ar', '21/11/2018', 'recebidos', 'os', 'autos', '19/11/2018', 'remetidos', 'os', 'autos', 'assinatura', 'para', 'magistrado',

In [9]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
nltk.download('rslp')
dicionario = set()
stemmer = nltk.stem.RSLPStemmer()

[nltk_data] Downloading package rslp to /home/jovyan/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [11]:
#pattern = "[A-Za-záàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ]+"
pattern = "[0-9]+"
dicionario = set()
for lista in textos_quebrados:
    sem_numeros=[]
    validas = [stemmer.stem(palavra) for palavra in lista if palavra not in stopwords and len(palavra)>2]
    for palavra in validas:
        result = re.search(pattern, palavra)
        if result is None:
            sem_numeros.append(palavra)
    #atualiza dicionario com todas as palavras
    dicionario.update(sem_numeros)

In [12]:
total_de_palavras = len(dicionario)
print (total_de_palavras)
print (dicionario)
#criando de x para de palavras e numeros
tuplas = zip(dicionario, range(total_de_palavras))

16846
{'computando-s', 'higid', 'asil', 'retoqu', 'identif', 'sales', 'perciv', 'epifani', 'precisas.entretant', 'implicit', 'esclareç', '/ou', 'desincumb', 'zein', 'kawab', 'internet', 'recint', 'jaceguay', 'i.p.c.', 'prolat', 'aguardem-s', 'aconteceu.p', 'pois.p', 'stj', 'convers', 'ingressando-s', 'resxt', 'dianary', 'instância.', 'executada.transit', 'condiz', 'sentenc', 'cabíveis.n', 'ary', 'storniol', 'lourenç', 'pignaton', 'sane', 'prospera.adem', 'ag.praz', 'banco.dev', 'estad', 'nenhum', 'intrínsec', "'cláusul", 'autos.v', 'interlocutóri', 'simples', 'perfilh', 'yank', 'legal-', 'destaque-s', 'processuais.p.i.c', 'apoem', 'argü', 'protest', 'lege.oportun', 'zabagl', '¿apel', 'murill', 'www.tjba.jus.br', 'sambr', 'reforç', 'interlocut', 'jan', 'protelatóri', 'gsofsm', 'peticionár', 'fracar', 'autorizados.cust', 'adél', 'lunard', 'selic', 'zacarin', 'imprensa.int', 'r.i', 'ulian', "'em", 'deisy', 'menequin', 'custas.arapirac', 'petica', 'sachett', 'amig', 'robal', 'falec', 'deci

In [13]:
#criando dicionario => traduz palavras em indices
tradutor = {palavra:indice for palavra,indice in tuplas}

In [14]:
def vetorizar_texto(texto, tradutor):
    #vetorizando frase - lentradutor qtde total de palavras
    vetor = [0] * len(tradutor)

    for palavra in texto:
        if len(palavra) > 0:
            raiz = stemmer.stem(palavra)
            if raiz in tradutor:
                posicao = tradutor[raiz]
                vetor[posicao] += 1
    return vetor

In [ ]:
#para cada texto do textos quebrados chama a funcao vetorizar texto e retorna array de todas as vetorizacoes
%time vetores_de_texto = [vetorizar_texto(texto, tradutor) for texto in textos_quebrados]

In [ ]:
marcas = classificacoes['status']
#classificacao

X = np.array(vetores_de_texto)
Y = np.array(marcas.tolist())

print(X)

In [ ]:
print(Y)

In [ ]:
porcentagem_de_treino = 0.8
tamanho_do_treino = int(porcentagem_de_treino * len(Y))
tamanho_de_validacao = len(Y) - tamanho_do_treino

treino_dados = X[0:tamanho_do_treino]
treino_marcacoes = Y[0:tamanho_do_treino]

validacao_dados = X[tamanho_do_treino:]
validacao_marcacoes = Y[tamanho_do_treino:]

In [ ]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = k)
    taxa_de_acerto = np.mean(scores)
    msg = "Taxa de acerto do {0}: {1}".format(nome, taxa_de_acerto)
    print (msg)
    return taxa_de_acerto

In [ ]:
def teste_real(modelo, validacao_dados, validacao_marcacoes):
    resultado = modelo.predict(validacao_dados)

    acertos = resultado == validacao_marcacoes

    total_de_acertos = sum(acertos)
    total_de_elementos = len(validacao_marcacoes)

    taxa_de_acerto = 100.0 * total_de_acertos / total_de_elementos

    msg = "Taxa de acerto do vencedor entre os dois algoritmos no mundo real: {0}".format(taxa_de_acerto)
    print(msg)

In [ ]:
resultados = {}
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state = 0))
resultadoOneVsRest = fit_and_predict("OneVsRest", modeloOneVsRest, treino_dados, treino_marcacoes)
resultados[resultadoOneVsRest] = modeloOneVsRest



In [ ]:
from sklearn.multiclass import OneVsOneClassifier
modeloOneVsOne = OneVsOneClassifier(LinearSVC(random_state = 0))
resultadoOneVsOne = fit_and_predict("OneVsOne", modeloOneVsOne, treino_dados, treino_marcacoes)
resultados[resultadoOneVsOne] = modeloOneVsOne

In [ ]:
from sklearn.naive_bayes import MultinomialNB
modeloMultinomial = MultinomialNB()
resultadoMultinomial = fit_and_predict("MultinomialNB", modeloMultinomial, treino_dados, treino_marcacoes)
resultados[resultadoMultinomial] = modeloMultinomial

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
modeloAdaBoost = AdaBoostClassifier(random_state=0)
resultadoAdaBoost = fit_and_predict("AdaBoostClassifier", modeloAdaBoost, treino_dados, treino_marcacoes)
resultados[resultadoAdaBoost] = modeloAdaBoost

In [ ]:
print (resultados)

maximo = max(resultados)
vencedor = resultados[maximo]

print ("Vencerdor: ")
print (vencedor)

In [ ]:
vencedor.fit(treino_dados, treino_marcacoes)

teste_real(vencedor, validacao_dados, validacao_marcacoes)

acerto_base = max(Counter(validacao_marcacoes).values())
taxa_de_acerto_base = 100.0 * acerto_base / len(validacao_marcacoes)
print("Taxa de acerto base: %f" % taxa_de_acerto_base)

total_de_elementos = len(validacao_dados)
print("Total de teste: %d" % total_de_elementos)
